In [1]:
import pandas as pd
from nltk.stem.cistem import Cistem
import re
from nltk.probability import FreqDist
from tabulate import tabulate
from tqdm import tqdm

# load fancy nlp pipeline :)))
import spacy

# WE NEED SPEED :(
# spacy.require_gpu()

nlp = spacy.load("de_core_news_lg")

import pickle
from pathlib import Path

out_path = Path("../data/frequencies_per_year_raw_data.pkl")

OSError: [E050] Can't find model 'de_core_news_lg'. It doesn't seem to be a Python package or a valid path to a data directory.

# Load data

In [8]:
data = pd.read_csv('../data/tagesschau_articles_unique.csv', sep='\t', engine='python', on_bad_lines='warn')
data.head()

,date,headline,short_headline,short_text,article,link
0,2025-10-23,++ Kiew: Übergabe von 1.000 Leichen durch Russ...,Krieg gegen die Ukraine,\n Ukrainische Behö...,Ukrainische Behörden haben die Übergabe von 1....,/newsticker/liveblog-ukraine-donnerstag-512.html
1,2025-10-23,Zehntausende bei Demonstrationen in Ungarn,Wahlkampf-Auftakt,\n Im Frühjahr wähl...,Im Frühjahr wählt Ungarn ein neues Parlament. ...,/ausland/europa/ungarn-proteste-132.html
2,2025-10-23,Litauen meldet Eindringen von russischen Flugz...,Top-Thema,\n Zwei russische M...,Zwei russische Militärflugzeuge sind nach Anga...,/eilmeldung/eilmeldung-8954.html
3,2025-10-23,"""Gemeinsam russische U-Boote jagen""",Deutsch-britische Zusammenarbeit,\n Großbritannien u...,Großbritannien und Deutschland wollen gemeinsa...,/ausland/europa/deutschland-grossbritannien-se...
4,2025-10-23,Wetterlage und Temperaturen,Wettervorhersage Europa,\n Sturmtief JOSHUA...,Sturmtief JOSHUA zieht bis Freitagabend mit se...,/wetter/europa-welt


In [9]:
# convert date column to pandas date format
data['date'] = pd.to_datetime(data['date'])

# Some of the articles are missing nan, because its only a video link
drop_indices = data[data['article'].isnull()].index
data = data.drop(index=drop_indices).reset_index(drop=True)

per_year = data.groupby(data['date'].dt.year)

counts_per_year = per_year.size()
print(counts_per_year)

date
2018     1409
2019     1382
2020     1630
2021     1119
2022      961
2023      966
2024     3705
2025    10889
dtype: int64


In [10]:
per_year.get_group(2023).head()

,date,headline,short_headline,short_text,article,link
14594,2023-12-31,++ ZDF-Mitarbeiterin bei Angriff auf Charkiw v...,Krieg gegen die Ukraine,\n Bei den russisch...,Bei den russischen Angriffen auf Charkiw ist e...,/newsticker/liveblog-ukraine-sonntag-374.html
14595,2023-12-31,++ Israel lässt einige Reservisten nach Hause ++,Krieg in Nahost,\n Einige Reservist...,Einige Reservisten der israelischen Streitkräf...,/newsticker/liveblog-israel-sonntag-126.html
14596,2023-12-31,Drei weitere Verdächtige in Gewahrsam,Terror-Warnung in Köln,\n In Zusammenhang ...,In Zusammenhang mit möglichen Anschlagsplänen ...,/inland/inland-koeln-dom-anschlagsgefahr-100.html
14597,2023-12-31,"Welkom Belgien, adiós Spanien",Wechsel der EU-Ratspräsidentschaft,\n Turnusgemäß wech...,Turnusgemäß wechselt der EU-Ratsvorsitz alle s...,/ausland/europa/eu-ratspraesidentschaft-spanie...
14598,2023-12-31,Königin Margrethe II. kündigt Abdankung an,Dänemark,\n Seit dem Tod von...,Seit dem Tod von Queen Elizabeth II. galt sie ...,/ausland/europa/daenemark-koenigin-margrethe-a...


In [11]:
cistem = Cistem()

def remove_temporary(msg: str) -> str:
    text = re.sub(r'[^a-zA-Z ]', ' ', msg)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def stem(msg: str) -> str:
    words = msg.split(' ')
    singles = [cistem.stem(word) for word in words]
    return ' '.join(singles)

def get_words_of_interest(text, types=["VERB", "ADJ", "AUX"]):
    doc = nlp(text)
    
    result = {}
    
    for type in types:
        result[type] = [remove_temporary(stem(token.lemma_)) for token in doc if token.pos_ == type]
    
    return result

get_words_of_interest(data['article'][0])

{'VERB': ['meld',
  'tot',
  'meld',
  'beschliess',
  'meld',
  'nenn',
  'meld',
  'meld',
  'eindring',
  'feststell',
  'teil',
  'teil',
  'aufstieg',
  'patrouillier',
  'geh',
  'geh',
  'geb',
  'seh',
  'entscheid',
  'absag',
  'verschieb',
  'sag',
  'sprech',
  'brem',
  'vorberei',
  'beto',
  'vorschlag',
  'zustimm',
  'auss',
  'vorwerf',
  'helf',
  'sag',
  'helf',
  'unterstreich',
  'geb',
  'zusich',
  'liefer',
  'geb',
  'lass',
  'beeintrachtig',
  'mach',
  'erklar',
  'stark',
  'war',
  'find',
  'brauch',
  'hoff',
  'hoff',
  'sag',
  'setz',
  'ubergeb',
  'ubergeb',
  'erfolg',
  'teil',
  'erklar',
  'beginn',
  'bedank',
  'erhal',
  'bestatig',
  'ubergeb',
  'erhal',
  'berg',
  'reagier',
  'komm',
  'sag',
  'geb',
  'geb',
  'zeig',
  'umsetz',
  'geh',
  'erhoh',
  'einstell',
  'komm',
  'sag',
  'komm',
  'vermiss',
  'tot',
  'erklar',
  'aufklar',
  'fug',
  'einlief',
  'mach',
  'nenn',
  'handel',
  'ford',
  'bitt',
  'ermoglich',
  'ford'

In [12]:
# Convert all articles
frequencies_per_year_raw_data = {}
frequencies_per_year = {}

for group, entry in per_year:
    articles = entry["article"]

    verbs = []
    adj = []
    aux = []

    articles = tqdm(articles, desc=f"Processing year {group}")
    for article in articles:
        words = get_words_of_interest(article)

        verbs.extend(words["VERB"])
        adj.extend(words["ADJ"])
        aux.extend(words["AUX"])

    freq_verbs = FreqDist(verbs)
    freq_adj = FreqDist(adj)
    freq_aux = FreqDist(aux)

    frequencies_per_year[group] = {"VERB": freq_verbs, "ADJ": freq_adj, "AUX": freq_aux}
    frequencies_per_year_raw_data[group] = {"VERB": verbs, "ADJ": adj, "AUX": aux}
    
# Store raw data to file for later use
with out_path.open("wb") as f:
    pickle.dump(frequencies_per_year_raw_data, f, protocol=pickle.HIGHEST_PROTOCOL)

print("Data saved to", out_path)

Processing year 2018:   0%|          | 0/1409 [00:00<?, ?it/s]

Processing year 2025: 100%|██████████| 10889/10889 [17:04<00:00, 10.63it/s]


Data saved to ../data/frequencies_per_year_raw_data.pkl


In [13]:
print("Top frequencies per year:")
for year, freq in frequencies_per_year.items():
    verbs = freq['VERB']
    adj = freq['ADJ']
    aux = freq['AUX']
    
    print(f"Year {year}:")
    
    number = 20
    verbs_head = verbs.most_common(number)
    
    adj_head = adj.most_common(number)
    aux_head = aux.most_common(number)
    
    df_verbs = pd.DataFrame(verbs_head, columns=['Verb', 'Frequency'])
    print(tabulate(df_verbs, headers='keys', tablefmt='github', showindex=False))

    df_adj = pd.DataFrame(adj_head, columns=['Adj', 'Frequency'])
    print(tabulate(df_adj, headers='keys', tablefmt='github', showindex=False))

    df_aux = pd.DataFrame(aux_head, columns=['Aux', 'Frequency'])
    print(tabulate(df_aux, headers='keys', tablefmt='github', showindex=False))

Top frequencies per year:
Year 2018:
| Verb   |   Frequency |
|--------|-------------|
| sag    |        2758 |
| geb    |        2298 |
| geh    |        1488 |
| komm   |        1361 |
| mach   |        1172 |
| hab    |        1139 |
| seh    |         911 |
| steh   |         821 |
| bleib  |         719 |
| erklar |         699 |
| lass   |         630 |
| stell  |         619 |
| ford   |         589 |
| sprech |         581 |
| lte    |         569 |
| heiss  |         558 |
| find   |         550 |
| hal    |         532 |
| zeig   |         493 |
| lieg   |         484 |
| Adj           |   Frequency |
|---------------|-------------|
| europaisch    |        1322 |
| neu           |        1261 |
| gross         |        1072 |
| ander         |        1049 |
| deutsch       |         938 |
|               |         643 |
| weit          |         617 |
| russisch      |         533 |
| ers           |         502 |
| vergang       |         488 |
| politisch     |         474